In [1]:
import pandas as pd, json
from datetime import datetime
import os

In [2]:
print(f"Início do cálculo: {datetime.now()}")

Início do cálculo: 2025-07-18 13:27:35.271578


In [3]:
# Carregar config e resultados
with open('config.json') as f: config=json.load(f)
with open('validacao_resultado.json') as f: valid=json.load(f)
arq_val = valid['arquivos_validos']
print(f"Arquivos válidos: {len(arq_val)}")

Arquivos válidos: 0


In [4]:
# Funções de cálculo
def calc_basicas(df, id_exp):
    m = []
    for grupo in df['grupo'].unique():
        d = df[df['grupo'] == grupo]
        total = len(d)
        conv = (d['uso'] > 0).sum() / total * 100 if total else 0
        m.append({'id_experimento': id_exp, 'grupo': grupo, 'metrica': 'taxa_conversao', 'valor': round(conv, 2), 'timestamp': datetime.now().isoformat()})
        uso_med = d['uso'].mean()
        m.append({'id_experimento': id_exp, 'grupo': grupo, 'metrica': 'uso_medio', 'valor': round(uso_med, 2), 'timestamp': datetime.now().isoformat()})
        valor_med = d['valor'].mean()
        m.append({'id_experimento': id_exp, 'grupo': grupo, 'metrica': 'valor_medio', 'valor': round(valor_med, 2), 'timestamp': datetime.now().isoformat()})
    return pd.DataFrame(m)


def calc_lift(df_meta, id_exp, grupo_controle):
    lifts = []
    df_gc = df_meta[df_meta['grupo'] == grupo_controle]
    for met in df_gc['metrica']:
        base = df_gc[df_gc['metrica'] == met]['valor'].iloc[0]
        if base <= 0: continue
        for g in df_meta['grupo'].unique():
            if g == grupo_controle: continue
            val = df_meta[(df_meta['grupo'] == g) & (df_meta['metrica'] == met)]['valor'].iloc[0]
            lifts.append({'id_experimento': id_exp, 'grupo': g, 'metrica': f'lift_{met}', 'valor': round((val - base) / base * 100, 2), 'timestamp': datetime.now().isoformat()})
    return pd.DataFrame(lifts)

In [5]:
# Executar loop de cálculo
todas = []
for info in arq_val:
    nome = info['nome']
    df = pd.read_csv(f"/tmp/processing/temp/{nome}")
    id_exp = df['id_experimento'].iloc[0]
    print(f"Processando {nome} ({id_exp})")
    df_bas = calc_basicas(df, id_exp)
    df_lft = calc_lift(df_bas, id_exp, config['grupo_controle'])
    df_full = pd.concat([df_bas, df_lft], ignore_index=True)
    df_full['arquivo_origem'] = nome
    todas.append(df_full)

if todas:
    df_all = pd.concat(todas, ignore_index=True)
    df_all.to_csv('metricas_completas.csv', index=False)
    print(f"✓ Métricas completas salvas: {len(df_all)} linhas")
else:
    print("⚠️ Nenhuma métrica calculada")

⚠️ Nenhuma métrica calculada
